In [26]:
import pandas as pd
import numpy as np

dft = pd.read_csv("times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("distances_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)

In [31]:
len(dft.columns)

66